# This notebook is for PPG analysis and feature extraction



In [ ]:
import neurokit2 as nk
import pandas as pd
import numpy as np
from tqdm import tqdm

#inv gaussian for point process
def invgauss_fit(bool_peaks, get='hr'):
    peaks = np.where(bool_peaks==1)[0]
    waiting_times = np.array(peaks[1:]-peaks[:-1])
    frequencies = 1.0/waiting_times
    mu = waiting_times.mean()
    #sns.distplot(frequencies)
    theta = 1.0/(((frequencies - 1.0/mu).sum())/frequencies.shape[0])
    
    mean_arrival_rate = 1.0/mu + 1.0/theta
    std_arrival_rate = np.sqrt(1.0/(mu*theta) + 2.0/(theta*theta))
    std_arrival_time = np.sqrt(mu**3/theta)
    
    if get == 'hr':
        return mean_arrival_rate
    elif get=='hrv':
        return std_arrival_rate


#hr from point process
def inv_hr(bool_peaks):
    return invgauss_fit(bool_peaks, get='hr')


#hrv from point process
def inv_hrv(bool_peaks):
    return invgauss_fit(bool_peaks, get='hrv')


#hr from point process
def get_peaks_hr(PPG_rolled):
    sample_rate=100
    signals, info = nk.ppg_process(PPG_rolled, sampling_rate=sample_rate)
    peaks = signals["PPG_Peaks"]
    arrival_hr = inv_hr(peaks)
    return arrival_hr


#hrv from point process
def get_peaks_hrv(PPG_rolled):
    sample_rate=100
    signals, info = nk.ppg_process(PPG_rolled, sampling_rate=sample_rate)
    peaks = signals["PPG_Peaks"]
    arrival_hrv = inv_hrv(peaks)
    return arrival_hrv


#hr from NK
def get_peaks_hr_nk(PPG_rolled):
    sample_rate=100
    signals, info = nk.ppg_process(PPG_rolled, sampling_rate=sample_rate)
    peaks = signals["PPG_Peaks"]
    avg_hr_nk = signals["PPG_Rate"].mean()
    return avg_hr_nk


#hrv from NK
def get_peaks_hrv_nk(PPG_rolled):
    sample_rate=100
    signals, info = nk.ppg_process(PPG_rolled, sampling_rate=sample_rate)
    peaks = signals["PPG_Peaks"]
    std_hr_nk = signals["PPG_Rate"].std()
    return std_hr_nk


def get_hrv_all(PPG_rolled):
    sample_rate=100
    signals, info = nk.ppg_process(PPG_rolled, sampling_rate=sample_rate)
    hrv_all = nk.hrv(peaks, sampling_rate=100, show=False)
    return hrv_all

### Reading the PPG file

In [ ]:
data = pd.read_csv("D:/Google Drive/UVA_PhD/Projects/Structural Equation Modeling/03072021_going_raw.csv")

### Run functions on all sensors

In [ ]:
#list of all sensors
list_of_sensors = ["PPG1","PPG3"]

#window size: example 4min*60s*100 = 24000
w= 24000
tqdm.pandas()

for sensor in list_of_sensors:
    data[sensor+"_hr"] = data[sensor].rolling(window=w).progress_apply(get_peaks_hr)
    data[sensor+"_hrv"] = data[sensor].rolling(window=w).progress_apply(get_peaks_hrv)
    data[sensor+"_hr_nk"] = data[sensor].rolling(window=w).progress_apply(get_peaks_hr_nk)
    data[sensor+"_hrv_nk"] = data[sensor].rolling(window=w).progress_apply(get_peaks_hrv_nk)
    #data["zzzz"] = data[sensor].rolling(window=w).apply(get_hrv_all)

### Run feature extraciton with a for loop

In [ ]:
import math
import heartpy as hp
from tqdm import tqdm

length_of_all = data.shape[0]

j=0
duration = 24000 #datapoints

results_feat = []
results_feat_nk = []

for j in tqdm(range(length_of_all-duration)):
    windowed_data = data.iloc[j:j+duration]
    try:
        signals, info = nk.ppg_process(windowed_data["PPG3"].values, sampling_rate=sample_rate)
        hr = get_peaks_hr(signals["PPG_Peaks"].values)
        hrv = get_peaks_hrv(signals["PPG_Peaks"].values)
        hr_nk = get_peaks_hr_nk(signals["PPG_Peaks"].values)
        hrv_nk = get_peaks_hrv_nk(signals["PPG_Peaks"].values)
        all_hrv_feats = nk.hrv(signals["PPG_Peaks"], sampling_rate=100, show=False)

        results_feat.append({"row":j,"Timestamp":windowed_data["Timestamp"].iloc[duration-1],"PPG3_hr":hr,"PPG3_hrv":hrv,
                            "PPG3_hr_nk":hr_nk,"PPG3_hrv_nk":hrv_nk})
        all_hrv_feats["Timestamp"] = windowed_data["Timestamp"].iloc[duration-1]
        results_feat_nk.append(all_hrv_feats)
    except:
        pass
   

